# Electric Fixed Wing sUAS Design

### Introduction
The following will overview the design process for an electric fixed wing small unmanned aerial system (sUAS) as well as providing code for designing one. Due to the nature of the vehicle being designed, the operational conditions are assumed to be standard sea level. All units are SI unless otherwise noted.

In [1]:
# Setup

# Imports
import numpy as np
import matplotlib.pyplot as plt

# Constants
pi = 3.1415926

# Physical Constants
gravity = 9.80665               # m/s^2

# Standard Sea Level Atmosphere
density = 1.225                 # kg/m^3
temp_C = 15                     # C
temp_K = 288.2                  # K
pressure = 101325               # Pa
speed_of_sound = 340.3          # m/s
dynamic_viscosity = 1.789e-5    # kg/m/s
kinematic_viscosity = 1.460e-5  # m^2/s
gas_constant = 287.1            # J/kg/K
temp_lapse_rate = 0.0065        # K/m
molar_mass = 0.0289652          # kg/mol
ratio_of_specific_heats = 1.4   # unitless

### Requirements

Any aircraft design process begins by defining requirements. These will likely be adjusted in an iterative process as certain combinations of requirements are mutually exclusive given a certain budget or current available technology.

In [2]:
# Requirements

W_payload = 0.5 * gravity  # N, required payload weight
range = 20000  # m, required range


### Initial Sizing Estimate

The first step of aircraft design is an estimate of gross takeoff weight given by the following formula

\begin{gathered}
W_0 = \frac{W_p}{1-\frac{W_b}{W_0}-\frac{W_e}{W_0}} 
\end{gathered}

Where

\begin{gathered}
W_0 = \text{Gross Takeoff Weight} \\
W_p = \text{Payload Weight (All Non-Flight-Critical Mass)} \\
\frac{W_b}{W_0} = \text{Battery Weight Fraction} \\
\frac{W_e}{W_0} = \text{Empty Weight Fraction} \\
\end{gathered}

Payload weight is a requirement. Battery Weight Fraction is estimated based on the mission requirements. Empty weight fraction is estimated based on the Gross Takeoff Weight. However, since Gross Takeoff Weight is what we are solving for, Empty Weight Fraction and Gross Takeoff Weight must be estimated iteratively.

Source: Raymer 6th Ed. Chapter 3

##### Battery Weight Fraction Estimate

Note that unlike standard fuel-based aircraft, electric aircraft use a constant-weight battery. The following method of estimating Battery Weight Fraction is used.

For a low altitude electric sUAS, I will assume climb and landing make up a relatively small portion of flight time and can therefore be treated identically to cruise. Therefore the Battery Weight Fraction can be estimated using only the Range, Estimated L/D and Estimated Powertrain Efficiency

\begin{gathered}
\frac{W_b}{W_0} = \frac{g*R}{3600*e*\frac{L}{D}*n}
\end{gathered}

where

\begin{gathered}
g = \text{gravity (m/s/s)} \\
R = \text{Range (m)} \\
e = \text {Battery Specific Energy (W*hr/kg)} \\
\frac{L}{D} = \text{Lift to Drag Ratio} \\
n = \text{Efficiency } (n_{propeller} * n_{motor} * n_{esc} * n_{battery} * n_{wire})
\end{gathered}


Source: Raymer 6th Ed. Chapter 20 Pg 759

Propeller efficiency is a function of advance ratio, but for intitial sizing purposes we will assume it falls between 0.5 and 0.7.

See: https://m-selig.ae.illinois.edu/props/propDB.html

Wire efficiency is extremely high at such short lengths and is assumed to be 0.99. All other efficiencies are fairly high and have been roughly estimated as 0.9.

Specific Energy is estimated to be approximately 150 W*Hr/kg for RC Hobby Grade LiPo Batteries.



In [5]:
n_propeller = .6  # propeller efficiency
n_wire = .99  # wire effiency
n_motor = .9  # motor efficiency
n_esc = .9  # esc efficiency
n_batt = .9  # battery efficiency

n_total = n_propeller * n_motor * n_esc * n_batt * n_wire
print(f"Total Efficiency: {n_total:.3f}")

e = 150  # W*Hr/kg, RC Grade Lipo Specific Energy


Total Efficiency: 0.433


##### Empty Weight Fraction Estimate